You should have **models.py** and **analyis.py** in your pyhton working folder (download it here https://github.com/khalednasr/nn-numerosity-zero).

In [1]:
import pandas as pd
import numpy as np
import os

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
import seaborn as sns
from tqdm.notebook import tqdm, trange

import models, analysis

In [19]:
path = '/Users/mirckuz/Desktop/20230601_Hz09_casper_h2bcamp7f_7dpf_60Z_1hzvol_2P_1v2v3v4v5_processed/voxelizer_final_datasets'
# Configuration parameters
Qrange = np.array([0,1,2,3,4]) # range of tested numerosities
colors = ['red', 'orange', 'green', 'blue', 'purple'] # plotting colors for numerosities in Qrange
matplotlib.rcParams.update({'font.size': 7.65}) # plot font size

for file in os.listdir(path):
    arr = np.loadtxt(file, delimiter=",")
    C = arr[:,0]#.astype(float).astype(int)
    Q = arr[:,1]#.astype(float).astype(int)
    Hf = arr[:,2:]
    print('\nANALYSING '+file+'\n')

    # Find numorosity selective units (anova_cells) using a two-way ANOVA
    pN, pC, pNC = analysis.anova_two_way(Q, C, Hf)
    anova_cells = np.where((pN<0.01) & (pNC>0.01) & (pC>0.01))[0]
    R = Hf[:,anova_cells]

    # Calculate and plot ditribution of preferred numerosities
    pref_num = analysis.preferred_numerosity(Q, R)
    hist = [np.sum(pref_num==q) for q in Qrange]
    hist /= np.sum(hist)

    plt.figure(figsize=(3,3))
    plt.bar(Qrange, 100*hist, width=0.8, color=colors)
    plt.xlabel('Preferred Numerosity')
    plt.ylabel('Percentage of cells')
    print('Number of anova cells for '+file+' = %i (%0.2f%%)'%(len(anova_cells), 100*len(anova_cells)/Hf.shape[1]))

    # Calculate average tuning curve of each unit
    tuning_curves = analysis.average_tuning_curves(Q, R) 

    # Calculate population tuning curves for each preferred numerosity
    tuning_mat = np.array([np.mean(tuning_curves[:,pref_num==q], axis=1) for q in Qrange]) # one row for each pref numerosity
    tuning_err = np.array([np.std(tuning_curves[:,pref_num==q], axis=1) / np.sqrt(np.sum(pref_num==q)) # standard error for each point on each tuning curve
                        for q in Qrange])

    # Normalize population tuning curves to the 0-1 range
    tmmin = tuning_mat.min(axis=1)[:,None]
    tmmax = tuning_mat.max(axis=1)[:,None]
    tuning_mat = (tuning_mat-tmmin) / (tmmax-tmmin)
    tuning_err = tuning_err / (tmmax-tmmin) # scale standard error to be consistent with above normalization

    # Plot population tuning curves on linear scale
    plt.figure(figsize=(8,2.75))
    plt.subplot(1,2,1)
    for i, (tc, err) in enumerate(zip(tuning_mat, tuning_err)):
        plt.errorbar(Qrange, tc, err, color=colors[i])
        plt.xticks(Qrange)
    plt.xlabel('Numerosity')
    plt.ylabel('Normalized Neural Activity')

    # Plot population tuning curves on log scale
    plt.subplot(1,2,2)
    for i, (tc, err) in enumerate(zip(tuning_mat, tuning_err)):
        plt.errorbar(Qrange+1, tc, err, color=colors[i]) # offset x axis by one to avoid taking the log of zero
        plt.xscale('log', base=2)
        plt.gca().xaxis.set_major_formatter(ScalarFormatter())
        plt.xticks(ticks=Qrange+1, labels=Qrange)
    plt.xlabel('Numerosity')
    plt.ylabel('Normalized Neural Activity')


(240,)
(240,)
(240, 14584)
